In [248]:
import pandas as pd
pd.options.display.max_colwidth = 100
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import light_location_dict

In [160]:
level1_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level1.json")))

In [161]:
level2_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level2.json")))

In [162]:
level3_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level3.json")))

In [163]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [164]:
def json_to_dataframe(json_file):
    if "," in json_file[0]["name_node"]:
        level_df = pd.DataFrame(columns=["pattern","supp","conf", "time"])
        for i in json_file:
            for j in i["patterns"]:
                level_df.loc[level_df.shape[0]] = j
    else:
        level_df = pd.DataFrame(columns=["name_node","supp","conf"])
        for i in json_file:
            level_df.loc[level_df.shape[0]] = i
    return level_df

In [222]:
level2_df = json_to_dataframe(level2_json)
level2_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights,0.849315,0.856354,"{'134': [[['2013-11-12 03:30:03', '2013-11-12 ..."
1,Load_StatusMBALights->Load_StatusSensHeatPrntB...,0.980822,0.988950,"{'134': [[['2013-11-12 06:29:03', '2013-11-12 ..."
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
3,Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
4,Load_StatusMBALights>Load_StatusPlugLoadCoffee...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
...,...,...,...,...
723,Load_StatusApplianceOven->Load_StatusClothesWa...,0.276712,0.651613,"{'180': [[['2013-12-28 08:14:06', '2013-12-28 ..."
724,Load_StatusClothesWasher>Load_StatusDryerPower...,0.405479,0.896970,"{'180': [[['2013-12-28 14:30:05', '2013-12-28 ..."
725,Load_StatusClothesWasher|Load_StatusPlugLoadLR...,0.268493,0.632258,"{'180': [[['2013-12-28 14:30:05', '2013-12-28 ..."
726,Load_StatusPlugLoadLRBlueRay>Load_StatusDryerP...,0.279452,0.618182,"{'180': [[['2013-12-28 14:59:05', '2013-12-28 ..."


In [166]:
level3_df = json_to_dataframe(level3_json)
level3_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.698630,0.704420,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
1,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
2,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.701370,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
3,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
4,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
...,...,...,...,...
5767,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.707182,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5768,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.701370,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5769,Load_StatusPlugLoadHeatingPad->Load_StatusSens...,0.698630,0.704420,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5770,Load_StatusApplianceOven>Load_StatusPlugLoadVi...,0.276712,0.651613,"{'180': [[['2013-12-28 08:14:06', '2013-12-28 ..."


In [232]:
def filter_rule_indexes(dataframe):
    return_dict = {}
    level_3_check = len(re.findall('\*', dataframe.loc[0, 'pattern'])) > 0
    #filter follows rules in dataframe for level3: max 1 and level2: 0
    if level_3_check:
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) <= 1]
    else:
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) == 0]

    appliance_appliance_appliance_rule_indexes = []
    person_appliance_appliance_rule_indexes = []
    person_person_appliance_rule_indexes = []

    for index, row in dataframe.iterrows():
        tmp_floor_set = set()
        appliance_check_list = list()
        person_check_list = list()
        for col in set(re.findall('[\w_]+', row['pattern'])):
            person_check_list.append('SensHeat' in col)
            appliance_check_list.append('SensHeat' not in col)
            try:
                tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
            except:
                tmp_floor_set.add(light_location_dict(meta)[col][0])
        if '1stFloor' in tmp_floor_set and '2ndFloor' in tmp_floor_set and sum(appliance_check_list) >= 1:
            if sum(person_check_list) == 1:
                person_appliance_appliance_rule_indexes.append(index)
                if len(person_appliance_appliance_rule_indexes) > 0:
                    if level_3_check:
                        return_dict.update({'psn_app_app': person_appliance_appliance_rule_indexes})
                    else:
                        return_dict.update({'psn_app': person_appliance_appliance_rule_indexes})
            elif sum(person_check_list) == 2:
                person_person_appliance_rule_indexes.append(index)
                if len(person_person_appliance_rule_indexes) > 0:
                    return_dict.update({'psn_psn_app': person_person_appliance_rule_indexes})
            elif sum(person_check_list) == 0:
                appliance_appliance_appliance_rule_indexes.append(index)
                if len(appliance_appliance_appliance_rule_indexes) > 0:
                    if level_3_check:
                        return_dict.update({'app_app_app': appliance_appliance_appliance_rule_indexes})
                    else:
                        return_dict.update({'app_app': person_appliance_appliance_rule_indexes})
    return return_dict

In [220]:
level2_index_dict = filter_rule_indexes(level2_df)

                                               pattern      supp      conf  \
2    Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN  0.704110  0.709945   
3      Load_StatusMBALights>Load_StatusPlugLoadBlender  0.701370  0.707182   
4    Load_StatusMBALights>Load_StatusPlugLoadCoffee...  0.701370  0.707182   
10           Load_StatusMBALights>Load_StatusBA2Lights  0.701370  0.707182   
14     Load_StatusMBALights>Load_StatusSensHeatPrntBUP  0.841096  0.841096   
..                                                 ...       ...       ...   
722  Load_StatusApplianceOven>Load_StatusPlugLoadVi...  0.279452  0.666667   
724  Load_StatusClothesWasher>Load_StatusDryerPower...  0.405479  0.896970   
725  Load_StatusClothesWasher|Load_StatusPlugLoadLR...  0.268493  0.632258   
726  Load_StatusPlugLoadLRBlueRay>Load_StatusDryerP...  0.279452  0.618182   
727  Load_StatusPlugLoadVacuum>Load_StatusPlugLoadIron  0.142466  0.509804   

                                                  time  
2    {

In [ ]:
level2_index_dict

In [216]:
level3_index_dict = filter_rule_indexes(level3_df)

True


In [211]:
len(level3_index_dict['psn_app_app'])

1058

In [ ]:
level2_df.loc[0, 'pattern']

In [ ]:
level3_df_follow_restricted.loc[(level3_df_follow_restricted.loc[person_appliance_rule_indexes, "pattern"].str.findall('>').map(len) >= 2).index.tolist()]

In [ ]:
person_appliance_level3 = level3_df_follow_restricted.loc[person_appliance_rule_indexes]
person_appliance_level3.loc[(person_appliance_level3['pattern'].str.contains('LR')) & (person_appliance_level3['pattern'].str.contains('KitchenLight')) & (person_appliance_level3['pattern'].str.contains('PrntB'))]

In [ ]:
level2_df_follow_restricted.loc[level2_index_dict['psn_app_app']].loc[level2_df_follow_restricted.loc[level2_index_dict['psn_app_app'], 'pattern'].str.findall('BR2[\w]*>[\w]*ChildA').map(len) > 0]

In [233]:
def redundancy_filter_tool(dataframe=level3_df, regex_str='BR3[\w]*>[\w]*ChildB', select_index_dict='psn_app_app'):
    return dataframe.loc[filter_rule_indexes(dataframe)[select_index_dict]].loc[dataframe.loc[filter_rule_indexes(dataframe)[select_index_dict], 'pattern'].str.findall(regex_str).map(len) > 0]

In [247]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', select_index_dict='psn_app')

,pattern,supp,conf,time
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 06:14:03'], ['2013-11-12 05:59:03', '2013-11-12 06..."
213,Load_StatusPlugLoadToaster>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'134': [[['2013-11-12 06:19:03', '2013-11-12 06:30:03'], ['2013-11-12 06:29:03', '2013-11-12 06..."
253,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntBUP,0.838356,0.838356,"{'134': [[['2013-11-12 06:29:03', '2013-11-12 08:24:03'], ['2013-11-12 06:29:03', '2013-11-12 06..."
265,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildAUP,0.564384,0.564384,"{'134': [[['2013-11-12 18:09:02', '2013-11-12 20:29:02'], ['2013-11-12 18:29:02', '2013-11-12 19..."
270,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'134': [[['2013-11-12 18:09:02', '2013-11-12 20:29:02'], ['2013-11-12 18:29:02', '2013-11-12 19..."
334,Load_StatusBR2Lights>Load_StatusSensHeatPrntADOWN,0.567123,0.571823,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 21:59:02'], ['2013-11-12 18:29:02', '2013-11-12 22..."
341,Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 21:59:02'], ['2013-11-12 19:29:02', '2013-11-12 20..."
343,Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 21:59:02'], ['2013-11-12 19:29:02', '2013-11-12 20..."
368,Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 20:29:02'], ['2013-11-12 19:29:02', '2013-11-12 20..."
370,Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 20:29:02'], ['2013-11-12 19:29:02', '2013-11-12 20..."


In [245]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*\|[\w]*', select_index_dict='psn_app')

,pattern,supp,conf,time
156,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPr...,0.701370,0.707182,"{'134': [[['2013-11-12 06:14:03', '2013-11-12 ..."
264,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChi...,0.701370,0.701370,"{'134': [[['2013-11-12 18:09:02', '2013-11-12 ..."
269,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChi...,0.838356,0.838356,"{'134': [[['2013-11-12 18:09:02', '2013-11-12 ..."
301,Load_StatusSensHeatPrntADOWN|Load_StatusPlugLo...,0.698630,0.704420,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."
389,Load_StatusDRLights|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'134': [[['2013-11-12 19:14:02', '2013-11-12 ..."
390,Load_StatusDRLights|Load_StatusSensHeatChildBUP,0.701370,0.701370,"{'134': [[['2013-11-12 19:14:02', '2013-11-12 ..."
417,Load_StatusSensHeatChildBDOWN|Load_StatusBR2Li...,0.558904,0.563536,"{'291': [[['2014-04-18 17:58:47', '2014-04-18 ..."
523,Load_StatusSensHeatChildAUP|Load_StatusPlugLoa...,0.975342,0.975342,"{'134': [[['2013-11-12 00:04:03', '2013-11-12 ..."
530,Load_StatusSensHeatChildAUP|Load_StatusDRLights,0.843836,0.843836,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."
542,Load_StatusSensHeatChildAUP|Load_StatusPlugLoa...,0.698630,0.698630,"{'291': [[['2014-04-18 17:58:47', '2014-04-18 ..."
